## play around with simulation 2

In [92]:
import arlpy.uwapm as pm
from kaooi.simulation import simulation
import numpy as np
import xarray as xr
from dask.distributed import Client
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34711,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.79 GiB
Comm: tcp://127.0.0.1:46125,Total threads: 4
Dashboard: http://127.0.0.1:45393/status,Memory: 7.85 GiB
Nanny: tcp://127.0.0.1:36371,


In [104]:
# Example usage
lat1, lon1 = 23.34915, -158.56992  
lat2, lon2 = 45.95482, -130.00890 # Central Caldera
num_points = 1000

bathy = simulation.get_bathymetry_slice((lat1, lon1), (lat2, lon2), 50)

In [109]:
ssp = simulation.get_ssp_slice((lat1, lon1), (lat2, lon2), 50)

In [141]:
simulation.write_bty(bathy, 'sim_files/test')
simulation.write_ssp(ssp.fillna(1600)[25:26,0,:], 'sim_files/test')

In [142]:
simulation.write_env('sim_files/test', ssp[:,0,:].fillna(1600).compute())

In [10]:
bathy_arlpy = np.vstack((bathy.range*1000, bathy.values)).T
ssp_arlpy = np.vstack((ssp.depth, ssp[0,:,25])).T
ssp_arlpy = np.vstack((ssp_arlpy[:-2,:], np.array([7000, 1560])))

/home/jhrag/miniconda3/envs/kb/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 483.19 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [11]:
ocean_env = pm.create_env2d(
    depth=bathy_arlpy,
    soundspeed=ssp_arlpy,
    bottom_soundspeed=1600,
    bottom_density=1200,
    bottom_absorption=1.0,
    tx_depth=800,
    rx_range = 3e6,
    rx_depth = 500
)

In [12]:
pm.plot_env(ocean_env)

In [22]:
model = pm._Bellhop()
env_fn = model._create_env_file(ocean_env, 'R')
print(env_fn)

/tmp/tmp430464fh


In [17]:
rays = model._load_rays(env_fn)

In [18]:
rays

,angle_of_departure,surface_bounces,bottom_bounces,ray
0,-80.000000,3,3,"[[0.0, 800.0], [8.815744512891213, 750.0], [17..."
0,-76.734694,3,3,"[[0.0, 800.0], [11.786708319507744, 750.0], [2..."
0,-73.469388,3,3,"[[0.0, 800.0], [14.838663766711512, 750.0], [2..."
0,-70.204082,3,3,"[[0.0, 800.0], [17.995732319336554, 750.0], [3..."
0,-66.938776,3,3,"[[0.0, 800.0], [21.28515243225078, 750.0], [42..."
0,-63.673469,3,3,"[[0.0, 800.0], [24.738305362335264, 750.0], [4..."
0,-60.408163,3,3,"[[0.0, 800.0], [28.39203414258553, 750.0], [56..."
0,-57.142857,3,3,"[[0.0, 800.0], [32.29038063766803, 750.0], [64..."
0,-53.877551,4,4,"[[0.0, 800.0], [36.486924430143475, 750.0], [7..."
0,-50.612245,4,4,"[[0.0, 800.0], [41.0480006727727, 750.0], [82...."


In [19]:
rays_shallow = rays[rays['bottom_bounces'] < 10]
rays_shallow = rays_shallow[rays_shallow['surface_bounces'] < 10]

In [21]:
pm.plot_rays(rays_shallow, env=ocean_env)

## Arlpy toy environment

In [2]:
bathy = [
    [0, 30],    # 30 m water depth at the transmitter
    [300, 20],  # 20 m water depth 300 m away
    [1000, 25]  # 25 m water depth at 1 km
]

ssp = [
    [ 0, 1540],  # 1540 m/s at the surface
    [10, 1530],  # 1530 m/s at 10 m depth
    [20, 1532],  # 1532 m/s at 20 m depth
    [25, 1533],  # 1533 m/s at 25 m depth
    [30, 1535]   # 1535 m/s at the seabed
]

In [8]:
env = pm.create_env2d(
    depth=bathy,
    soundspeed=ssp,
    bottom_soundspeed=1450,
    bottom_density=1200,
    bottom_absorption=1.0,
    tx_depth=15
)

In [11]:
pm.plot_env(env)

In [12]:
model = pm._Bellhop()
env_fn = model._create_env_file(env, 'REIA')

In [13]:
env_fn

'/tmp/tmpyzvc9pkj'